<a href="https://colab.research.google.com/github/romelgo/Prediccion-de-la-perdida-de-clientes-mediante-Machine-Learning/blob/main/Web_Scraping_UPeU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proceso de ingestión de datos mediante web scraping en **upeu.edu.pe**:

**1. Objetivo**

Elegimos una universidad (**en este caso la UPEU)** porque su página web concentra información estructurada y relevante en distintos ámbitos: **oferta académica, docentes, líneas de investigación, noticias institucionales y servicios a la comunidad.**
El objetivo principal es ***construir un dataset realista y confiable*** que sirva de base para análisis educativos, institucionales y de innovación. Además, trabajar con una universidad permite obtener información pública (no privada ni sensible) que respeta la ética del scraping y que es útil para estudios académicos.

**2. Datos o información necesarios para recolectar**

Algunos de los datos más relevantes que se podrían extraer de la web de upeu.edu.pe son:

- Carreras profesionales y programas académicos (descripción, duración, malla curricular).

- Docentes e investigadores (nombres, grados académicos, especialidades, publicaciones).

- Noticias y eventos (temas recurrentes, comunicados institucionales).

- Convocatorias y admisión (fechas, requisitos, procesos).

- Información sobre campus (ubicaciones, facultades, servicios estudiantiles).

**Extrae y normaliza:**

Respetamos `robots.txt` automáticamente.

- Carreras y mallas curriculares.

- Docentes (enlaces a “Plana Docente” y, opcional, lectura tabular de PDF con Camelot).

- Noticias y eventos (título, fecha, categorías, url).

- Convocatorias y admisión (modalidades + requisitos).

- Información de campus (sedes y direcciones).

In [1]:
# =========================
# UPeU Web Scraping Toolkit
# (c) uso académico - respeta robots.txt
# =========================

# ---- Instalación de librerías (Colab) ----
!pip -q install requests beautifulsoup4 lxml pandas tqdm dateparser html5lib
# Camelot es opcional para leer tablas desde algunos PDFs de "Plana Docente"
# (requiere ghostscript; si no quieres leer PDFs tabulares, pon USE_PDF=False más abajo)
!pip -q install camelot-py[cv] ghostscript PyPDF2

# ---- Imports ----
import re, time, json, sys, math, io, itertools
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import urllib.robotparser as robotparser
from tqdm.auto import tqdm
import dateparser

# =========================
# Configuración general
# =========================
BASE = "https://upeu.edu.pe/"
NEWS_BASE = "https://upeu.edu.pe/noticias-upeu/"
MALLAS_URL = "https://upeu.edu.pe/malla-curricular-y-planes-academicos/"
PLANA_DOCENTE_URL = "https://upeu.edu.pe/transparencia/plana-docente/"
ADMISION_MODALIDADES = "https://admision.upeu.edu.pe/modalidades-de-ingreso/"
ADMISION_INSCRIPCION = "https://admision.upeu.edu.pe/inscripcion/"
MEDICINA_TRANSP = "https://medicina.upeu.edu.pe/transparencia/"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) UPeU-Scraper/1.0 (+educational use)"
}
REQUEST_TIMEOUT = 20
SLEEP_BETWEEN = 1.0   # Respeto básico a servidores (ajusta si hace falta)
MAX_NEWS_PAGES = 3    # N páginas de noticias a recorrer
USE_PDF = True        # Si True, intentará leer PDFs tabulares de “Plana Docente” con Camelot (best-effort)

# =========================
# Utilidades: robots, fetch, soup
# =========================
def allowed_by_robots(url: str, user_agent: str = HEADERS["User-Agent"]) -> bool:
    """
    Verifica permiso de crawl según robots.txt del dominio del URL.
    """
    parsed = urlparse(url)
    robots_url = f"{parsed.scheme}://{parsed.netloc}/robots.txt"
    rp = robotparser.RobotFileParser()
    try:
        rp.set_url(robots_url)
        rp.read()
        allowed = rp.can_fetch(user_agent, url)
        return True if allowed is None else allowed
    except Exception:
        # Si no hay robots o fallo de lectura, asumimos prudencia pero permitimos (sitios pequeños suelen carecer de robots)
        return True

def get(url, stream=False):
    """
    GET robusto con headers, timeout y control de robots.
    """
    if not allowed_by_robots(url):
        raise PermissionError(f"Bloqueado por robots.txt: {url}")
    resp = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT, stream=stream)
    resp.raise_for_status()
    return resp

def get_soup(url):
    """
    Devuelve BeautifulSoup de un URL (HTML).
    """
    time.sleep(SLEEP_BETWEEN)
    r = get(url)
    return BeautifulSoup(r.text, "lxml")

def clean_text(x: str) -> str:
    if x is None: return ""
    return re.sub(r"\s+", " ", x).strip()

# =========================
# 1) Carreras y Mallas Curriculares
# =========================
def scrape_mallas():
    """
    Extrae:
      - Facultad
      - Programa
      - Tipo (Pregrado/Posgrado/Especialidad, si se infiere del bloque)
      - URL del plan/malla (normalmente Google Drive/SharePoint)
    """
    soup = get_soup(MALLAS_URL)
    data = []
    # Heurística: recorrer encabezados y enlaces debajo
    current_block = None   # p.ej. "Pregrado", "Posgrado", "Especialidades"
    current_facultad = None
    for tag in soup.find_all(["h2", "h3", "h4", "h5", "p", "a"]):
        name = clean_text(tag.get_text())
        if tag.name in ("h2", "h3"):
            # Bloques grandes suelen decir "Pregrado", "Posgrado", "Especialidades"
            if re.search(r"pregrado|posgrado|especial", name, re.I):
                current_block = name
        if tag.name in ("h3", "h4") and re.search(r"ciencias|ingenier|teolog|arquitect", name, re.I):
            current_facultad = name
        if tag.name == "a":
            href = tag.get("href", "")
            text = clean_text(tag.get_text())
            if href and text and len(text) > 2:
                # Consideramos que es un programa si el enlace luce como malla/plan (a menudo GDrive o SharePoint)
                if re.search(r"(drive\.google\.com|sharepoint|\.pdf$)", href, re.I):
                    data.append({
                        "bloque": current_block,
                        "facultad": current_facultad,
                        "programa_o_documento": text,
                        "url": urljoin(MALLAS_URL, href)
                    })
    df = pd.DataFrame(data).drop_duplicates()
    return df

# =========================
# 2) Docentes e Investigadores
# =========================
def collect_plana_docente_links():
    """
    Recolecta enlaces desde:
      - Transparencia general (si expone links)
      - Transparencia de Medicina (ejemplo claro con PDFs por periodo)
    Heurística: enlaces cuyo texto u href contenga 'Docente' o 'Docentes' y (si hay) PDFs.
    """
    links = []

    # Página general
    try:
        soup = get_soup(PLANA_DOCENTE_URL)
        for a in soup.select("a"):
            txt = clean_text(a.get_text())
            href = a.get("href", "")
            if re.search(r"docente", txt, re.I) or re.search(r"docente", href, re.I):
                links.append({"origen": "Plana Docente (Transparencia)", "texto": txt, "url": urljoin(PLANA_DOCENTE_URL, href)})
    except Exception as e:
        print("Aviso: no se pudieron extraer enlaces desde Plana Docente general:", e)

    # Ejemplo Medicina (demostrativo, suele listar PDFs de “Plana Docentes 20xx-x”)
    try:
        soup_m = get_soup(MEDICINA_TRANSP)
        for a in soup_m.select("a"):
            txt = clean_text(a.get_text())
            href = a.get("href", "")
            if re.search(r"docente", txt, re.I) or re.search(r"docente", href, re.I):
                links.append({"origen": "Transparencia Medicina", "texto": txt, "url": urljoin(MEDICINA_TRANSP, href)})
    except Exception as e:
        print("Aviso: no se pudieron extraer enlaces desde Transparencia Medicina:", e)

    df = pd.DataFrame(links).drop_duplicates()
    # Filtramos URLs vacías y normalizamos
    df = df[df["url"].astype(bool)]
    return df.reset_index(drop=True)

def try_parse_docentes_pdf(pdf_url):
    """
    BEST-EFFORT: intenta leer tablas desde un PDF con Camelot (puede fallar si el PDF no es tabular).
    Retorna DataFrame con columnas genéricas.
    """
    import camelot
    rows = []
    try:
        # Descarga temporal del PDF
        time.sleep(SLEEP_BETWEEN)
        r = get(pdf_url, stream=True)
        content = r.content
        tmp_path = "/tmp/plana_docente.pdf"
        with open(tmp_path, "wb") as f:
            f.write(content)
        # Intento de lectura mixta (stream y lattice)
        tables = []
        try:
            tables += camelot.read_pdf(tmp_path, pages="all", flavor="lattice")
        except Exception:
            pass
        try:
            tables += camelot.read_pdf(tmp_path, pages="all", flavor="stream")
        except Exception:
            pass
        for t in tables:
            df_t = t.df
            # Limpieza básica: primeras filas a menudo son headers
            df_t.columns = [f"col_{i}" for i in range(df_t.shape[1])]
            for _, row in df_t.iterrows():
                rows.append(row.to_dict())
        if rows:
            out = pd.DataFrame(rows)
            out["source_pdf"] = pdf_url
            return out
    except Exception as e:
        # Si falla, retornamos vacío
        return pd.DataFrame()
    return pd.DataFrame()

def scrape_docentes():
    """
    Devuelve:
      - listado de enlaces “plana docente” (períodos, facultad/origen)
      - (opcional) concatenación de tablas extraídas de PDFs si USE_PDF=True
    """
    links_df = collect_plana_docente_links()
    parsed_tables = []
    if USE_PDF and not links_df.empty:
        # Intentaremos solo los que apunten a .pdf (para no perseguir GDrive privados)
        pdf_links = links_df[links_df["url"].str.contains(r"\.pdf", case=False, na=False)]
        # Dedicamos un límite razonable para demo (p.ej. hasta 3 PDFs)
        for pdf_url in tqdm(pdf_links["url"].head(3), desc="Leyendo PDFs (demo)"):
            df_pdf = try_parse_docentes_pdf(pdf_url)
            if not df_pdf.empty:
                parsed_tables.append(df_pdf)
    docentes_df = pd.concat(parsed_tables, ignore_index=True) if parsed_tables else pd.DataFrame()
    return links_df, docentes_df

# =========================
# 3) Noticias y Eventos
# =========================
def parse_news_article(url):
    """
    Extrae título, fecha (si aparece), categorías/etiquetas y un extracto.
    """
    soup = get_soup(url)
    title = clean_text(soup.find(["h1","h2"]).get_text()) if soup.find(["h1","h2"]) else ""
    # Fechas en WordPress suelen estar en <time> o en un span con mes/día
    date_candidates = []
    for time_tag in soup.select("time"):
        date_candidates.append(clean_text(time_tag.get_text()))
    # Alternativo: buscar patrones como "9 Sep | Noticias Lima"
    meta_texts = [clean_text(x.get_text()) for x in soup.select("span, div") if re.search(r"\b\d{1,2}\s(de|[A-Za-z])\b", clean_text(x.get_text()))]
    date_str = date_candidates[0] if date_candidates else (meta_texts[0] if meta_texts else "")
    parsed_date = dateparser.parse(date_str, languages=["es"]) if date_str else None
    # Categorías: enlaces que contengan /category/ o “Noticias Lima/Juliaca/Tarapoto”
    cats = []
    for a in soup.select("a"):
        t = clean_text(a.get_text())
        href = a.get("href","")
        if re.search(r"/category/|Noticias\s+(Lima|Juliaca|Tarapoto)", t, re.I):
            cats.append(t)
    # Extracto
    p = soup.find("p")
    excerpt = clean_text(p.get_text()) if p else ""
    return {
        "titulo": title,
        "fecha_texto": date_str,
        "fecha_iso": parsed_date.isoformat() if parsed_date else "",
        "categorias": "; ".join(sorted(set(cats))),
        "url": url,
        "extracto": excerpt[:400]
    }

def scrape_news(max_pages=MAX_NEWS_PAGES):
    """
    Recorre el índice de noticias y sigue los enlaces a cada noticia.
    """
    all_items = []
    for page in range(1, max_pages+1):
        idx_url = NEWS_BASE if page == 1 else urljoin(NEWS_BASE, f"page/{page}/")
        soup = get_soup(idx_url)
        # Enlaces a artículos: heurística -> anchors que contengan "/noticias/"
        article_links = []
        for a in soup.select("a"):
            href = a.get("href","")
            # Evitar paginación duplicada
            if "/noticias/" in href and not href.endswith(("/category/","/tag/")):
                article_links.append(urljoin(idx_url, href))
        article_links = sorted(set(article_links))
        for link in tqdm(article_links, desc=f"Noticias página {page}", leave=False):
            try:
                all_items.append(parse_news_article(link))
            except Exception:
                pass
    df = pd.DataFrame(all_items).drop_duplicates(subset=["url"])
    return df

# =========================
# 4) Convocatorias y Admisión
# =========================
def scrape_admision():
    """
    Extrae modalidades y sus requisitos desde la página de modalidades de ingreso.
    """
    soup = get_soup(ADMISION_MODALIDADES)
    blocks = []
    # Patrón: títulos tipo "Examen ordinario", "Deportista calificado", etc., seguidos de listas <li>
    for h in soup.find_all(re.compile("^h[2-4]$")):
        title = clean_text(h.get_text())
        if not title:
            continue
        # Buscar items de lista cercanos (hermanos o descendientes próximos)
        ul = h.find_next(["ul","ol"])
        requisitos = []
        if ul:
            for li in ul.find_all("li"):
                requisitos.append(clean_text(li.get_text()))
        # Guardamos el bloque si parece una modalidad
        if re.search(r"examen|grado|t[ií]tulo|deportista|discapacidad|modalidad", title, re.I):
            blocks.append({"modalidad": title, "requisitos": "; ".join(requisitos)})
    df = pd.DataFrame(blocks).drop_duplicates()
    return df

# =========================
# 5) Información de Campus (direcciones, sedes)
# =========================
def scrape_campus():
    """
    Desde la portada principal extrae nombres de campus y direcciones que se listan en la sección 'Nuestros Campus'.
    """
    soup = get_soup(BASE)
    items = []
    # Heurística: encabezados con 'Campus' y el siguiente texto conteniendo dirección/teléfono
    for hdr in soup.find_all(re.compile("^h[3-5]$")):
        txt = clean_text(hdr.get_text())
        if re.search(r"^Campus\s", txt):
            # Buscar párrafo o líneas siguientes
            addr = []
            for sib in itertools.islice(hdr.next_siblings, 0, 6):
                if getattr(sib, "get_text", None):
                    t = clean_text(sib.get_text())
                    if t and not re.search(r"^\s*$", t) and len(t) < 200:
                        addr.append(t)
            address = "; ".join(addr)
            items.append({"campus": txt, "detalle": address})
    df = pd.DataFrame(items).drop_duplicates()
    return df

# =========================
# Orquestación principal
# =========================
def main():
    # 1) Mallas / planes
    print(">> Extrayendo mallas y planes académicos…")
    mallas_df = scrape_mallas()
    mallas_df.to_csv("upeu_mallas_planes.csv", index=False, encoding="utf-8-sig")
    print(f"Guardado: upeu_mallas_planes.csv ({len(mallas_df)} filas)")

    # 2) Docentes
    print(">> Recolectando enlaces de 'Plana Docente'…")
    enlaces_df, docentes_pdf_df = scrape_docentes()
    enlaces_df.to_csv("upeu_docentes_enlaces.csv", index=False, encoding="utf-8-sig")
    print(f"Guardado: upeu_docentes_enlaces.csv ({len(enlaces_df)} filas)")
    if USE_PDF and not docentes_pdf_df.empty:
        docentes_pdf_df.to_csv("upeu_docentes_pdf_tablas.csv", index=False, encoding="utf-8-sig")
        print(f"Guardado: upeu_docentes_pdf_tablas.csv ({len(docentes_pdf_df)} filas)")
    else:
        print("Aviso: no se extrajeron tablas desde PDF (puede que los PDFs no sean tabulares o requieran autenticación).")

    # 3) Noticias
    print(f">> Extrayendo noticias (primeras {MAX_NEWS_PAGES} páginas)…")
    noticias_df = scrape_news(max_pages=MAX_NEWS_PAGES)
    noticias_df.to_csv("upeu_noticias.csv", index=False, encoding="utf-8-sig")
    print(f"Guardado: upeu_noticias.csv ({len(noticias_df)} filas)")

    # 4) Admisión
    print(">> Extrayendo modalidades y requisitos de admisión…")
    adm_df = scrape_admision()
    adm_df.to_csv("upeu_admision_modalidades.csv", index=False, encoding="utf-8-sig")
    print(f"Guardado: upeu_admision_modalidades.csv ({len(adm_df)} filas)")

    # 5) Campus
    print(">> Extrayendo información de campus…")
    campus_df = scrape_campus()
    campus_df.to_csv("upeu_campus.csv", index=False, encoding="utf-8-sig")
    print(f"Guardado: upeu_campus.csv ({len(campus_df)} filas)")

    # Muestras
    print("\n=== EJEMPLOS DE SALIDA ===")
    for name, df in [
        ("MALLAS/PLANES", mallas_df.head(10)),
        ("DOCENTES (ENLACES)", enlaces_df.head(10)),
        ("NOTICIAS", noticias_df.head(5)),
        ("ADMISIÓN", adm_df.head(10)),
        ("CAMPUS", campus_df.head(10)),
    ]:
        print(f"\n-- {name} --")
        display(df)

# Ejecutar
main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.4 MB/s eta 0:00:00
>> Extrayendo mallas y planes académicos…
Guardado: upeu_mallas_planes.csv (74 filas)
>> Recolectando enlaces de 'Plana Docente'…


Leyendo PDFs (demo):   0%|          | 0/2 [00:00<?, ?it/s]

Guardado: upeu_docentes_enlaces.csv (16 filas)
Guardado: upeu_docentes_pdf_tablas.csv (231 filas)
>> Extrayendo noticias (primeras 3 páginas)…


Noticias página 1:   0%|          | 0/14 [00:00<?, ?it/s]

Noticias página 2:   0%|          | 0/19 [00:00<?, ?it/s]

...

,titulo,fecha_texto,fecha_iso,categorias,url,extracto
0,"UPeU supera los 1,000 artículos en SCOPUS y co...",Beca 18TransparenciaReavivados por su palabraP...,,Noticias Juliaca; Noticias Lima; Noticias Tara...,https://upeu.edu.pe/noticias/author/prensaupeu/,"by prensaupeu | Sep 10, 2025 | Noticias Juliac..."
1,"UPeU supera los 1,000 artículos en SCOPUS y co...",,,Noticias Juliaca; Noticias Lima; Noticias Tara...,https://upeu.edu.pe/noticias/category/juliaca/,"by prensaupeu | Sep 10, 2025 | Noticias Juliac..."
2,"UPeU supera los 1,000 artículos en SCOPUS y co...",Beca 18TransparenciaReavivados por su palabraP...,,Noticias Juliaca; Noticias Lima; Noticias Tara...,https://upeu.edu.pe/noticias/category/noticias/,"by prensaupeu | Sep 10, 2025 | Noticias Juliac..."
3,"UPeU supera los 1,000 artículos en SCOPUS y co...",,,Noticias Juliaca; Noticias Lima; Noticias Tara...,https://upeu.edu.pe/noticias/category/tarapoto/,"by prensaupeu | Sep 10, 2025 | Noticias Juliac..."
4,Docentes de Arquitectura de la UPeU proponen d...,,,Noticias Lima; Noticias Tarapoto,https://upeu.edu.pe/noticias/docentes-de-arqui...,"by prensaupeu | Ago 18, 2025 | Noticias Lima, ..."



-- ADMISIÓN --


,modalidad,requisitos
0,Examen ordinario,Certificado original de estudios secundarios; ...
1,Examen ordinario,Certificado de estudios secundarios o bachille...
2,Examen ordinario (Semipresencial o A Distancia),Certificado original de estudios secundarios; ...
3,Grado o Título,Certificado de estudios secundarios originales...
4,Grado o Título,Certificado de estudios secundarios originales...
5,Deportista calificado,Certificado original de estudios secundarios; ...
6,Persona con discapacidad,Certificado original de estudios secundarios; ...



-- CAMPUS --


,campus,detalle
0,Campus Universitarios,con infraestructura moderna y extensas áreas v...
1,Campus Lima,"Km 19 Carretera Central, Ñaña, Lurigancho, Lim..."
2,Campus Juliaca,"Salida a Arequipa Km. 6 Chullunquiani, Autopis..."
3,Campus Tarapoto,"Jr. Los Mártires Nro. 340, Morales, Tarapoto, ..."
